# Entity Embedding Tutorial

In this tutorial, we walk through how to generate Bootleg contextual entity embeddings for use in downstream tasks using a pretrained Bootleg model. We also demonstrate how to extract Bootleg's static learned embeddings for downstream tasks when contextualized embeddings are not needed.

### Requirements

You will need to download the following files for this notebook:
- The sample of Natural Questions with hand-labelled entities [here](https://bootleg-emb.s3.amazonaws.com/data/nq.tar.gz)
- Entity profile information [here](https://bootleg-emb.s3.amazonaws.com/entity_db.tar.gz)
- Pretrained Bootleg model and config [here](https://bootleg-emb.s3.amazonaws.com/models/2020_08_25/bootleg_wiki.tar.gz)
- Embedding data [here](https://bootleg-emb.s3.amazonaws.com/emb_data.tar.gz)

These are the same files as the End-to-End tutorial and do not need to be re-downloaded if you completed that tutorial. 

For convenience, you can run the command below (from the `tutorials` directory) to download all the above files and unpack them to the provided directory. It will take several minutes to download all the files. 

    bash download_all.sh <NAME_OF_DIRECTORY_TO_SAVE_DATA>
    
You will need to assign the variable `input_dir` in this notebook to the path where you download the data.  


## 1.  Prepare Model Config

As with the other tutorials, we set up the config to point to the correct data directories and model checkpoint. We use the sample of [Natural Questions](https://ai.google.com/research/NaturalQuestions) with mentions extracted by Bootleg introduced in the End-to-End tutorial. 

In [2]:
import numpy as np 
import pandas as pd
import ujson
from utils import load_mentions, tagme_annotate

from bootleg import run
from bootleg.utils.parser_utils import get_full_config

# set up logging
import sys
import logging
from importlib import reload
reload(logging)
logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

If you do not have a GPU with at least 12GB of memory available, set the below to `True` to run inference on the CPU. 

In [13]:
use_cpu = False

We need to specify the input directory where files were downloaded below. 

In [14]:
input_dir = # FILL IN WITH FULL PATH TO DATA DIRECTORY WHERE FILES ARE DOWNLOADED

config_path = f'{input_dir}/bootleg_wiki/bootleg_config.json'
config_args = get_full_config(config_path)

# set the model checkpoint path 
config_args.run_config.init_checkpoint = f'{input_dir}/bootleg_wiki/bootleg_model.pt'

# set the path for the entity db and candidate map
config_args.data_config.entity_dir = f'{input_dir}/entity_db'
config_args.data_config.alias_cand_map = 'alias2qids_wiki.json'

# set the data path and RSS500 test file 
config_args.data_config.data_dir = f'{input_dir}/nq'

# to speed things up for the tutorial, we have already prepped the data with the mentions detected by Bootleg
config_args.data_config.test_dataset.file = 'test_natural_questions_50_bootleg.jsonl'

# set the embedding paths 
config_args.data_config.emb_dir =  f'{input_dir}/emb_data'
config_args.data_config.word_embedding.cache_dir =  f'{input_dir}/emb_data'

# set the save directory 
config_args.run_config.save_dir = f'{input_dir}/results'

# set whether to run inference on the CPU
config_args.run_config.cpu = use_cpu

## 2. Load Contextual Entity Embeddings

We now show how Bootleg contextualized embeddings can be loaded and used in downstream tasks. First we use the `dump_embs` mode to generate contextual entity embeddings. 

In [6]:
bootleg_label_file, bootleg_emb_file = run.model_eval(args=config_args, mode="dump_embs", logger=logger, is_writer=True)

2020-09-09 13:09:37,243 PyTorch version 1.5.0 available.
2020-09-09 13:09:38,540 Loading entity_symbols...
2020-09-09 13:10:24,499 Loaded entity_symbols with 5222808 entities.
2020-09-09 13:10:24,505 Loading slices...
2020-09-09 13:10:24,507 Finished loading slices.
2020-09-09 13:10:24,509 Loading dataset...
2020-09-09 13:10:24,512 Finished loading dataset.
2020-09-09 13:10:38,605 Sampled 50 indices from dataset (dev/test) for evaluation.
2020-09-09 13:10:38,824 Loading embeddings...
2020-09-09 13:11:01,853 Finished loading embeddings.
2020-09-09 13:11:01,901 Loading model from data/bootleg_wiki/bootleg_model.pt...
2020-09-09 13:11:04,343 Successfully loaded model from data/bootleg_wiki/bootleg_model.pt starting from checkpoint epoch 2 and step 0.
2020-09-09 13:11:04,360 ************************DUMPING PREDICTIONS FOR test_natural_questions_50_bootleg.jsonl************************
2020-09-09 13:11:04,490 64 samples, 2 batches
2020-09-09 13:11:09,572 Writing predictions...
2020-09-09 13

In `dump_embs` mode, Bootleg saves the contextual entity embeddings corresponding to each mention in each sentence to a file. We return this file in the variable `bootleg_emb_file`. We can also see the full file path in the log (ends in `*npy`). 

In [7]:
import numpy as np
contextual_entity_embs = np.load(bootleg_emb_file)
contextual_entity_embs.shape

(100, 350)

Each row in the contextual entity embedding above corresponds to an extracted mention in a sentence. In the above embedding there are 100 extracted mentions total with 350 dimensions for each corresponding contextual entity embedding.

The mapping from mentions to rows in the contextual entity embedding is stored in `ctx_emb_ids` in the label file. We now check out the label file, which was also generated and returned from running `dump_embs` mode.

In [8]:
import jsonlines
with jsonlines.open(bootleg_label_file) as f: 
    for i, line in enumerate(f): 
        print('sentence:', line['sentence'])
        print('mentions:', line['aliases'])
        print('contextual emb ids:', line['ctx_emb_ids'])
        print()
        if i == 5: 
            break

sentence: who did the voice of the magician in frosty the snowman
mentions: ['the voice', 'the magician', 'frosty the snowman']
contextual emb ids: [0, 1, 2]

sentence: what is considered the outer banks in north carolina
mentions: ['outer banks', 'north carolina']
contextual emb ids: [3, 4]

sentence: the nashville sound brought a polished and cosmopolitan sound to country music by
mentions: ['the nashville sound', 'cosmopolitan', 'country music']
contextual emb ids: [5, 6, 7]

sentence: what channel is the premier league on in france
mentions: ['premier league', 'france']
contextual emb ids: [8, 9]

sentence: i love it ( feat . charli xcx ) icona pop
mentions: ['i love it', 'charli xcx', 'icona pop']
contextual emb ids: [10, 11, 12]

sentence: the u.s. supreme court hears appeals from circuit courts
mentions: ['us supreme court', 'circuit courts']
contextual emb ids: [13, 14]



In the first sentence, we can find the corresponding contextual entity embedding for "the voice", "the magician", and "frosty the snowman" in rows 0, 1, and 2 of `contextual_entity_embs`, respectively. Similarly, we have unique row ids for the mentions in each of the other sentences. A downstream task can use this process to load the correct contextual entity embeddings for each mention in a simple dataloader.

## 3. Load Static Entity Embeddings

In addition to contextual entity embeddings, Bootleg learns static entity embeddings. These can be useful in downstream tasks when contextual information is not available for the downstream task, or if we want the same entity embedding regardless of the context or position of the mention.

We walk through how to extract the static, learned entity embeddings from a pretrained Bootleg model. First, we define a utility function to load a model.

In [11]:
import os 
import torch 

from bootleg.model import Model
from bootleg.symbols.entity_symbols import EntitySymbols
from bootleg.utils import data_utils

def load_model(config_args, device='cuda', logger=None):
    logger.info(f'Using device {device}')
    entity_db =  EntitySymbols(os.path.join(config_args.data_config.entity_dir,
                                                             config_args.data_config.entity_map_dir), 
                              alias_cand_map_file=config_args.data_config.alias_cand_map)
    word_db = data_utils.load_wordsymbols(config_args.data_config, is_writer=True, distributed=False)

    model = Model(args=config_args, model_device=device,
            entity_symbols=entity_db, word_symbols=word_db).to(device)
    
    logger.info(f'Loading model from {config_args.run_config.init_checkpoint}.')
    model_state_dict = torch.load(config_args.run_config.init_checkpoint,
            map_location=lambda storage, loc: storage)['model']
    logger.info('Loaded model.')
    model.load_state_dict(model_state_dict, strict=True)
    model.eval()
    return model

Load the pretrained Bootleg model. This will take several minutes. 

In [15]:
model = load_model(config_args, logger=logger, device='cuda' if not use_cpu else 'cpu')

2020-09-09 13:16:34,411 Using device cuda
2020-09-09 13:17:40,255 Loading embeddings...
2020-09-09 13:18:09,154 Finished loading embeddings.
2020-09-09 13:18:12,417 Loading model from data/bootleg_wiki/bootleg_model.pt.
2020-09-09 13:18:14,394 Loaded model.


Get the static, learned entity embedding as a torch tensor. 

In [12]:
ent_embedding = model.emb_layer.entity_embs.learned.learned_entity_embedding.weight.data 
ent_embedding.shape

torch.Size([5222810, 200])

This Bootleg model was trained on data with 5.2 million entities and each entity embedding is 200-dimensional, as indicated by the shape of the static, learned entity embedding above.

The mapping from mentions to rows in the static, learned entity embedding (corresponding to the predicted entity) is also saved in the label file produced by `dump_embs` mode. We check out the label file below and use the `entity_ids` key to find the corresponding embedding row.  

In [13]:
import jsonlines
with jsonlines.open(bootleg_label_file) as f: 
    for i, line in enumerate(f): 
        print('sentence:', line['sentence'])
        print('mentions:', line['aliases'])
        print('entity ids:', line['entity_ids'])
        print()
        if i == 5: 
            break

sentence: who did the voice of the magician in frosty the snowman
mentions: ['the voice', 'the magician', 'frosty the snowman']
entity ids: [3071527, 45732, 306516]

sentence: what is considered the outer banks in north carolina
mentions: ['outer banks', 'north carolina']
entity ids: [647415, 9799]

sentence: the nashville sound brought a polished and cosmopolitan sound to country music by
mentions: ['the nashville sound', 'cosmopolitan', 'country music']
entity ids: [4738654, 23349, 2164]

sentence: what channel is the premier league on in france
mentions: ['premier league', 'france']
entity ids: [4919, 315399]

sentence: i love it ( feat . charli xcx ) icona pop
mentions: ['i love it', 'charli xcx', 'icona pop']
entity ids: [3485924, 3363307, 3469283]

sentence: the u.s. supreme court hears appeals from circuit courts
mentions: ['us supreme court', 'circuit courts']
entity ids: [14506, 30881]



Unlike the contextual entity embeddings, the static embeddings are not unique across mentions. For instance, if the same entity is predicted across two different mentions, the static entity embedding (and ids in the label file) will be the same for those mentions, whereas the contextual entity embeddings and ids will be different. 